In [1]:
import numpy as np
import pandas as pd
import numpy as np
import pandas as pd

## Important Note on Barcodes
- barcode sequences are drawn randomly from the pool of valid barcodes
- During aggregation, cellranger appends an integer to each barcode to keep them unique.
- However, since we used a custom aggregation script, which didn't append this integer until the cells were filtered, the original raw_counts.mtx rowdata does not have unique barcodes in this sense.
- for upload to single cell portal, we will therefore make the barcodes unique for both the raw and QC data

## First, some quick checks:
quick check that all the barcodes are unique within an individual:


QCed data

In [2]:
import pandas as pd
qc_row_data = pd.read_csv('./sc_portal_data/qc_rowData.csv')
out = []
for sample in np.unique(qc_row_data['sample_id']):
    i = qc_row_data[qc_row_data['sample_id']==sample]['barcodes']
    temp = [x.split('_')[0] for x in i]
    print(sample, len(np.unique(temp))==len(temp))
    out.append(len(np.unique(temp))==len(temp))

print('all samples have unique barcodes:', all(out))

D17-8756 True
D17-8758 True
D17-8766 True
D17-8768 True
D17-8772 True
D17-8778 True
D17-8786 True
D17-8790 True
D17-8792 True
D17-8796 True
D20-7432 True
D20-7433 True
D20-7434 True
D20-7435 True
D20-7436 True
D20-7437 True
D20-7438 True
D20-7439 True
D20-7440 True
D20-7441 True
D20-7442 True
D20-7443 True
D20-7444 True
D20-7445 True
D20-7446 True
D20-7447 True
D20-7448 True
D20-7450 True
D20-7451 True
D20-7452 True
D20-7453 True
D20-7454 True
D20-7455 True
D20-7456 True
D20-7457 True
D20-7458 True
all samples have unique barcodes: True


raw data

In [3]:
import pandas as pd
raw_row_data = pd.read_csv('./sc_portal_data/raw_rowData.csv')
barcodes = np.load('./sc_portal_data/barcodes.npy')
out = []
for sample in np.unique(raw_row_data['libraryID']):
    i = barcodes[raw_row_data['libraryID']==sample]
    print(sample, len(np.unique(i))==len(i))
    out.append(len(np.unique(i))==len(i))

print('all samples have unique barcodes:', all(out))

D17-8754 True
D17-8756 True
D17-8758 True
D17-8760 True
D17-8764 True
D17-8766 True
D17-8768 True
D17-8772 True
D17-8776 True
D17-8778 True
D17-8782 True
D17-8786 True
D17-8790 True
D17-8792 True
D17-8796 True
D17-8800 True
D20-7432 True
D20-7433 True
D20-7434 True
D20-7435 True
D20-7436 True
D20-7437 True
D20-7438 True
D20-7439 True
D20-7440 True
D20-7441 True
D20-7442 True
D20-7443 True
D20-7444 True
D20-7445 True
D20-7446 True
D20-7447 True
D20-7448 True
D20-7450 True
D20-7451 True
D20-7452 True
D20-7453 True
D20-7454 True
D20-7455 True
D20-7456 True
D20-7457 True
D20-7458 True
all samples have unique barcodes: True


All the raw data as the same suffix, as we did not aggregate with cellranger

In [4]:

x = qc_row_data['barcodes']
temp = np.array([i.split('_')[0].split('-')[1] for i in x])
for sample in np.unique(qc_row_data['sample_id']):
    d = np.unique(temp[qc_row_data['sample_id']==sample])
    print(sample, d)

D17-8756 ['1']
D17-8758 ['1']
D17-8766 ['1']
D17-8768 ['1']
D17-8772 ['1']
D17-8778 ['1']
D17-8786 ['1']
D17-8790 ['1']
D17-8792 ['1']
D17-8796 ['1']
D20-7432 ['1']
D20-7433 ['1']
D20-7434 ['1']
D20-7435 ['1']
D20-7436 ['1']
D20-7437 ['1']
D20-7438 ['1']
D20-7439 ['1']
D20-7440 ['1']
D20-7441 ['1']
D20-7442 ['1']
D20-7443 ['1']
D20-7444 ['1']
D20-7445 ['1']
D20-7446 ['1']
D20-7447 ['1']
D20-7448 ['1']
D20-7450 ['1']
D20-7451 ['1']
D20-7452 ['1']
D20-7453 ['1']
D20-7454 ['1']
D20-7455 ['1']
D20-7456 ['1']
D20-7457 ['1']
D20-7458 ['1']


Make raw data barcodes unique by combining barcode & seq library

In [194]:
# transferred raw_counts.mtx from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/open_data/synapse/expression_matrices/raw_counts.mtx (uploaded to single cell portal)  
barcodes = np.load('./sc_portal_data/barcodes.npy') # copied from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/raw_data/cellranger_counts_out/aggr_out/
metadata = np.load('./sc_portal_data/metadata.npy', allow_pickle=True)  # copied from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/raw_data/cellranger_counts_out/aggr_out/
rowData = pd.read_csv("./sc_portal_data/raw_rowData.csv") # from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/open_data/synapse/expression_matrices/
rowData['barcodes'] = barcodes
unique_barcodes_raw = rowData['libraryID'] + '-' + rowData['barcodes'] # make barcodes unique
print('check correspondence')
print(np.unique(metadata[:,2]==rowData['libraryID']))

# check uniqueness of barcodes
print(len(np.unique(unique_barcodes_raw))==len(unique_barcodes_raw))
# save them 
pd.DataFrame({'barcodes': unique_barcodes_raw}).to_csv('./sc_portal_data/barcodes_raw.csv', index=False)

check correspondence
[ True]
True


Make QC data barcodes unique by combining barcode & seq library

In [203]:
qc_rowData = pd.read_csv("./sc_portal_data/qc_rowData.csv") # from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/open_data/synapse/expression_matrices/
qc_rowData_barcodes = qc_rowData['sample_id'] + '-' +[x.split('_')[0] for x in qc_rowData['barcodes']]
print(len(np.unique(qc_rowData_barcodes))==len(qc_rowData_barcodes))
pd.DataFrame({'barcodes': qc_rowData_barcodes}).to_csv('./sc_portal_data/barcodes_qc.csv', index=False)

True


Check that all qc barcodes have a counterpart in raw data

In [196]:
i = set(unique_barcodes_raw)
qc_in_raw = [x in i for x in qc_rowData_barcodes]
print('all qc barcodes in raw data:', all(qc_in_raw))


all qc barcodes in raw data: True


### make required standard scp metadata file - only for processed / filtered data / logcounts

tudies uploaded using the SCP Metadata convention must include the following required metadata:

Metadata name	Content	Description
NAME*	string	unique identifier for each cell in the study. Must be the first column in a "Classic" format TSV metadata file.
*unneeded for AnnData files, adata.obs_names supplies the required NAME data 
biosample_id	string	unique identifier for each sample in the study
donor_id	string	unique identifier for each biosample donor in the study
species	ontology	ontology identifier from NCBItaxon
species__ontology_label	ontology_label	ontology label from NCBItaxon
disease	ontology	ontology identifier from MONDO or PATO (if no disease, use ontology ID "PATO_0000461")
disease__ontology_label	ontology_label	ontology label from MONDO or PATO (if no disease, use ontology label "normal")
organ	ontology	ontology identifier from Uberon
organ__ontology_label	ontology_label	ontology label from Uberon
library_preparation_protocol	ontology	ontology identifier from Experimental Factor ontology:library preparation
library_preparation_protocol__ontology_label	ontology_label	ontology label from Experimental Factor ontology:library preparation
sex	controlled list (enum)	one of ["male", "female", "mixed", "unknown"]
Required ontology metadata must include corresponding ontology labels with column header <ontology>__ontology_label to enable validation.

In [220]:
barcodes_raw = pd.read_csv('./sc_portal_data/barcodes_raw.csv')
barcodes_raw.columns = ['NAME']
barcodes_raw['biosample_id'] = ['-'.join(x.split('-')[:2]) for x in barcodes_raw['NAME']]
barcodes_raw['donor_id'] = ['-'.join(x.split('-')[:2]) for x in barcodes_raw['NAME']]
barcodes_raw['species__ontology_label'] = 'Homo sapiens'
barcodes_raw['species'] = 'NCBITaxon:9606'
barcodes_raw['organ__ontology_label'] = 'Brodmann (1909) area 10'
barcodes_raw['organ'] = 'UBERON:0013541'

summary = pd.read_csv('/Users/djuna/Documents/ABCA7lof2/processed_data/single_cell_individual_metadata.csv')
summary.index = summary['sample_id']
library_dictionary = {'JBM':'EFO:0009922', '171013Tsa':'EFO:0009899'}
library_dictionary2 = {'JBM':"10x 3' v3", '171013Tsa':"10x 3' v2"}
summary['library_preparation_protocol'] = summary['seq_batch'].map(library_dictionary)
summary['library_preparation_protocol__ontology_label'] = summary['seq_batch'].map(library_dictionary2)
summary['sex'] = summary['msex'].map({1:'male', 0:'female'})
summary['disease'] = summary['niareagansc'].map({0: 'MONDO:0004975', 1: 'MONDO:0004975', 2: 'PATO:0000461', 3: 'PATO:0000461'})
summary['disease__ontology_label'] = summary['niareagansc'].map({0: 'Alzheimer disease', 1: 'Alzheimer disease', 2: 'normal', 3: 'normal'})
summary['important_note_disease__ontology_label'] = 'disease__ontology_label based on niareagansc; if niareagansc is 0, 1, disease__ontology_label is Alzheimer disease; if niareagansc is 3, 4 disease__ontology_label is normal.'
summary_barcodes_raw = summary.loc[barcodes_raw['donor_id']]

barcodes_raw.index = barcodes_raw['donor_id']
scp_metadata = pd.concat([barcodes_raw, summary_barcodes_raw], axis=1)
scp_metadata = scp_metadata[['NAME', 'biosample_id', 'donor_id', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label', 'sex']]


In [221]:
scp_metadata

,NAME,biosample_id,donor_id,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label,sex
D20-7432,D20-7432-AAACCCAAGCATGATA-1,D20-7432,D20-7432,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009922,10x 3' v3,male
D20-7432,D20-7432-AAACCCAAGTAAACGT-1,D20-7432,D20-7432,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009922,10x 3' v3,male
D20-7432,D20-7432-AAACCCACAGAATTCC-1,D20-7432,D20-7432,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009922,10x 3' v3,male
D20-7432,D20-7432-AAACGAAAGCACGGAT-1,D20-7432,D20-7432,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009922,10x 3' v3,male
D20-7432,D20-7432-AAACGAACATTCACAG-1,D20-7432,D20-7432,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009922,10x 3' v3,male
...,...,...,...,...,...,...,...,...,...,...,...,...
D17-8786,D17-8786-TTTGCGCCAGCGTTCG-1,D17-8786,D17-8786,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009899,10x 3' v2,female
D17-8786,D17-8786-TTTGCGCGTACCATCA-1,D17-8786,D17-8786,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009899,10x 3' v2,female
D17-8786,D17-8786-TTTGCGCTCTTACCTA-1,D17-8786,D17-8786,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009899,10x 3' v2,female
D17-8786,D17-8786-TTTGGTTAGACACGAC-1,D17-8786,D17-8786,NCBITaxon:9606,Homo sapiens,PATO:0000461,normal,UBERON:0013541,Brodmann (1909) area 10,EFO:0009899,10x 3' v2,female


In [222]:
# filter out non-annotated cells
scp_metadata.index = scp_metadata['NAME']
qc_barcodes = pd.read_csv("./sc_portal_data/barcodes_qc.csv")
temp = scp_metadata.loc[qc_barcodes['barcodes']]

values = np.concatenate((np.repeat('TYPE', 1), np.repeat('group', 11)))
first_row = pd.DataFrame([values])
first_row.columns = scp_metadata.columns
scp_metadata = pd.concat([first_row, temp], ignore_index=True, axis=0)


scp_metadata.to_csv('./sc_portal_data/filtered_scp_metadata.csv',  index=False)

Additional cluster metadata (for filtered cells only)

In [180]:
qc_rowData = pd.read_csv("./sc_portal_data/qc_rowData.csv") # from  djuna@txe1-login.mit.edu:/home/gridsan/djuna/homer/github/ABCA7lof2/open_data/synapse/expression_matrices/
qc_barcodes = pd.read_csv("./sc_portal_data/barcodes_qc.csv")
qc_rowData['X'] = qc_rowData['x']
qc_rowData['Y'] = qc_rowData['y']



names = ['X', 'Y', 'total_counts', 'mito_fractions', 
         'nft', 'amyloid',  
          'seq_batch', 'sample_id',  'msex', 
         'ceradsc', 'braaksc', 'cogdx', 
         'niareagansc', 'ad_reagan',
         'apoe_genotype', 'ABCA7LoF', 
          'age_death_redacted', 
          'pmi_redacted', 'annotations']

values = np.concatenate((np.repeat('TYPE', 1), np.repeat('numeric', 6), np.repeat('group', 13))).astype(str)


In [181]:
qc_rowData_subset = qc_rowData[names]
qc_rowData_subset['NAME'] = qc_barcodes['barcodes'].copy()
cols = ['NAME'] + [col for col in qc_rowData_subset.columns if col != 'NAME']
qc_rowData_subset = qc_rowData_subset[cols]

first_row = pd.DataFrame([values])
first_row.columns = qc_rowData_subset.columns
qc_rowData_subset = pd.concat([first_row, qc_rowData_subset], ignore_index=True, axis=0)

/var/folders/y7/zpj5hbtd2bl78b1t9h2723hw0000gn/T/ipykernel_60270/809213100.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  qc_rowData_subset['NAME'] = qc_barcodes['barcodes'].copy()


In [182]:
qc_rowData_subset.to_csv("./sc_portal_data/filtered_cells_metadata.csv", index=False)

DEGs

In [175]:
degs = pd.read_csv('../extended_tables/supplementary_table_6.csv')